In [1]:
import pandas as pd
import numpy as np

# coo_matrix efektywny format rzadkich macierzy
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
import json

#Zarządzanie pamięcią
import gc
import os
from tqdm import tqdm

from config import *

from utils import calculate_map_at_k

# Ustawienie liczby wątków cpu na 1 dla stabliności treningu
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"


C:\Users\bart_\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt
def visualize_interactions_per_user(train_df):
    interactions = train_df.groupby('user_id')['item_id'].count()

    plt.figure(figsize=(8, 5))
    plt.hist(interactions, bins=50, color="purple", log=True)
    plt.xlabel("Liczba interakcji")
    plt.ylabel("Liczba użytkowników (log)")
    plt.title("Rozkład interakcji na użytkownika")
    plt.tight_layout()
    plt.savefig("interactions_per_user.png")
    plt.close()


In [3]:
def visualize_most_popular_items(train_df):
    popular_items = train_df.groupby('item_id')['rating'].count().sort_values(ascending=False).head(10)

    plt.figure(figsize=(8, 5))
    plt.barh([str(item) for item in popular_items.index[::-1]], popular_items.values[::-1], color="skyblue")
    plt.xlabel("Liczba ocen")
    plt.title("Top 10 najczęściej ocenianych przedmiotów (cold start)")
    plt.tight_layout()
    plt.savefig("cold_start_popular_items.png")
    plt.close()


In [4]:
def load_and_prepare_data():
    print("[ETAP 1/3] Wczytywanie i przetwarzanie danych")

    # Wczytanie danych z plików CSV.
    train_data_frame = pd.read_csv(TRAIN_PATH)
    test_data_frame = pd.read_csv(TEST_PATH)

    # Zliczamy unikalnych użytkowników z train i test dla nowej macierzy
    unique_users = np.unique(np.concatenate([train_data_frame['user_id'].unique(), test_data_frame['user_id'].unique()]))

    # Zliczamy unikalne przedmioty z train dla nowej macierz
    unique_items = train_data_frame['item_id'].unique()

    # Towrzymy odpowiedniej wymiary nowej macierz
    num_users = len(unique_users)
    num_items = len(unique_items)

    # Utworzenie zmapowanych id itemtów i userów (łatwiejsze liczenie w macierzy) od 0 do N a nie od 0 , 1 ,2 , 8 , 30
    user_to_idx = {user: i for i, user in enumerate(unique_users)}
    item_to_idx = {item: i for i, item in enumerate(unique_items)}

    idx_to_item = {i: item for item, i in item_to_idx.items()}

    # Mapowanie surowych ID na ciągłe indexy w nowej macierzy
    train_data_frame['user_idx'] = train_data_frame['user_id'].map(user_to_idx)
    train_data_frame['item_idx'] = train_data_frame['item_id'].map(item_to_idx)

    # Usunięcie wierszy, które nie mogły zostać zmapowane Pod cold start (aby nie było błędów przy czytaniu użytkowników których nie było przy treningu)
    train_data_frame.dropna(subset=['user_idx', 'item_idx'], inplace=True)
    train_data_frame[['user_idx', 'item_idx']] = train_data_frame[['user_idx', 'item_idx']].astype(int)

    # Mapowanie końcowych ID użytkowników w danych testowych
    test_data_frame['user_idx'] = test_data_frame['user_id'].map(user_to_idx)
    test_data_frame.dropna(subset=['user_idx'], inplace=True)
    test_data_frame['user_idx'] = test_data_frame['user_idx'].astype(int)

    print(f"Całkowita liczba unikalnych użytkowników (zmapowanych): {num_users}")
    print(f"Całkowita liczba unikalnych przedmiotów (zmapowanych): {num_items}")

    gc.collect() # Zwolnienie pamięci po usunięciach

    # train_data_frame / test_data_frame - przetworzone stare tabele już na nowo zindexowane
    # user_to_idx / item_to_idx - możemy odczytać orginalne ID na nowe indexy
    # idx_to_item - z indexu na orginalne ID
    # num_users / num_items - nasze wymiary macierzy

    # Dodaj w load_and_prepare_data przed return
    # I to dodaj również
    visualize_interactions_per_user(train_data_frame)
    visualize_most_popular_items(train_data_frame)
    return train_data_frame, test_data_frame, user_to_idx, item_to_idx, idx_to_item, num_users, num_items

In [5]:
def train_and_validate_als(params, val_users, ground_truth, user_item_matrix, user_to_idx, idx_to_item):

    print(f"Parametry: factors={params['factors']}, regularization={params['regularization']}, iterations={params['iterations']}")

    # Inicjalizujemy model ALS z parametrami z configu
    model_als = AlternatingLeastSquares(factors=params['factors'], regularization=params['regularization'], iterations=params['iterations'], random_state=ALS_RANDOM_STATE )

    # Trening modelu na naszej macierzy użytkownikowo x przedmiotowej
    model_als.fit(user_item_matrix)

    # Generujemy rekomendacje dla każdego użytkownika naszej walidacji
    als_recommendations_val = {}

    for user_raw_id in tqdm(val_users, desc=f"Walidacja ({params['name']})"):

        user_idx = user_to_idx[user_raw_id]

        # Bierzemy top 10 N rekomendacji z najwyższymi wynikami
        recommendations_idx, _ = model_als.recommend(user_idx, user_item_matrix[user_idx], N=10, filter_already_liked_items=True)
        # Zamieniamy indexy na surowe ID
        als_recommendations_val[user_raw_id] = [idx_to_item[rec_idx] for rec_idx in recommendations_idx]

    # Liczby MAP@10 z naszego utils
    map_score = calculate_map_at_k(als_recommendations_val, ground_truth, k=10)

    print(f"Wynik MAP@10 dla '{params['name']}': {map_score:.5f}")

    # Zwalniamy pamięć po validacji modelu
    del model_als

    gc.collect()

    # Zwracamy wynik
    return map_score

In [6]:
def train_final_model_and_predict(best_params, full_train_df, test_df, user_to_idx, idx_to_item, num_users, num_items):

    print("\n[ETAP 3/3 + Ostateczny trening] Trenowanie finalnego modelu i dawanie wyniku csv")
    print(f"Używam parametrów: factors={best_params['factors']}, regularization={best_params['regularization']}, iterations={best_params['iterations']}")

    # Budowanie pełnej macierzy interakcji z całych danych treningowych
    full_user_item_matrix_als = coo_matrix((full_train_df['rating'], (full_train_df['user_idx'], full_train_df['item_idx'])), shape=(num_users, num_items) ).tocsr()

    # Inicjalizacja ostatecznego modelu ALS
    model_als_final = AlternatingLeastSquares(factors=best_params['factors'], regularization=best_params['regularization'],iterations=best_params['iterations'], random_state=ALS_RANDOM_STATE )

    #Trening modelu na naszej ostatecznej macierzy
    model_als_final.fit(full_user_item_matrix_als)

    gc.collect()

    if not SKIP_SUBMISSION:
        # Zapisy dla wszystkich użytkowników MAP@10
        submission_output = {}

        # Bierzemy tylko użytkowników z test
        users_for_prediction_raw_ids = test_df['user_id'].unique()

        # Lecimy po wszystkich użytkownikach
        for user_raw_id in tqdm(users_for_prediction_raw_ids, desc="Generowanie csv"):
            # Bierzemy orginalny ID użytkownika
            user_idx = user_to_idx.get(user_raw_id)

            if user_idx is not None:
                # Patrzymy jakie przedmioty użytkownik już widział aby nie polecisz tego samego
                user_interactions = full_user_item_matrix_als[user_idx]

                # Generujemy top 10 rekomendacji filtrując te co widział
                recs_idx, _ = model_als_final.recommend(user_idx, user_interactions, N=10, filter_already_liked_items=True)
                top_10_item_ids = [idx_to_item.get(rec_idx, 0) for rec_idx in recs_idx]

                # Upewniamy się że na pewno jest to 10 ocen przedmiotów
                if len(top_10_item_ids) < 10:
                    top_10_item_ids.extend([0] * (10 - len(top_10_item_ids)))

                # Zapisujemy plik
                submission_output[user_raw_id] = " ".join(map(str, top_10_item_ids))

                # Jeśli nie ma użytkownika odpalamy cold start
            else: submission_output[user_raw_id] = DEFAULT_COLD_START_PREDICTIONS

        # Tworzymy i zapisujemy nowy plik subbmision według wzoru
        submission_df_final = pd.DataFrame({'user_id': users_for_prediction_raw_ids})
        submission_df_final['predictions'] = submission_df_final['user_id'].map(submission_output)
        submission_df_final['predictions'] = submission_df_final['predictions'].fillna(DEFAULT_COLD_START_PREDICTIONS)

        submission_df_final.to_csv(SUBMISSION_FILE, index=False)
        print(f"\nPlik submission zapisany do {SUBMISSION_FILE}")

    else: print("Generowanie pliku pominięte")

In [7]:
def main():

    # Wczytujemy potrzebne dane funkcja load_and_prepare_data

    train_df, test_df, user_to_idx, item_to_idx, idx_to_item, num_users, num_items = load_and_prepare_data()

    # Ustawiamy najlepsze parametry na brak dla pewnosóci
    best_params = None

    # Patrzymy czy w configu wybraliśmy trening tylko jednego modelu na jednym parametrze
    if SINGLE_MODEL_TRAINING and len(ALS_CONFIG) == 1:

        # Ładujemy pierwszy element parametru (czyli jedyny)
        best_params = ALS_CONFIG[0]

        print(f"\n[ETAP 3/3] Trenujemy tylko jeden model z jednym zbiorem parametrów")

        # Patrzymy czy w configu mamy ustawione skipowanie validacji dla opcjonalnego przyśpieszenia treningu
        if SKIP_VALIDATION:
            print("[ETAP 2/3] Wyłączona walidacja danych")

        else:
            print("\n[ETAP 2/3] Podział danych do walidacji")

            # Sotrujemy interakcje ocen w czasie (kolumna timestamp)
            train_df_sorted = train_df.sort_values(by='timestamp').reset_index(drop=True)

            # Wcześniej wybieramy ile chcemy mieć danych do validacji (podzial danych)
            split_point = int(len(train_df_sorted) * VALIDATION_SPLIT_RATIO)

            # Rozdzielenie danych na train i validation
            my_train_set_df = train_df_sorted.iloc[:split_point]
            my_validation_set_df = train_df_sorted.iloc[split_point:]

            # Czyścimy niepotrzebne dane których nie używamy
            del train_df_sorted
            gc.collect()

            # ! Bierzemy użytkowników ktorzy mieli interakcje i nie mili w przedziale czasowym
            users_in_validation_raw_ids = my_validation_set_df['user_id'].unique()
            users_in_train_for_validation_raw_ids = my_train_set_df['user_id'].unique()
            users_to_evaluate_raw_ids_full = np.intersect1d(users_in_validation_raw_ids, users_in_train_for_validation_raw_ids)

            # Zbieramy losowe dane
            np.random.seed(ALS_RANDOM_STATE)
            num_users_to_sample = int(len(users_to_evaluate_raw_ids_full) * (VALIDATION_SAMPLE_PERCENT / 100.0))
            users_to_evaluate_raw_ids = np.random.choice(users_to_evaluate_raw_ids_full, size=num_users_to_sample, replace=False)

            print(f"Zastosowano walidacje: {VALIDATION_SAMPLE_PERCENT}% użytkowników ({len(users_to_evaluate_raw_ids)} z {len(users_to_evaluate_raw_ids_full)})")

            # Mówmy jasno które elementy użytkownik ocenił
            ground_truth_val = my_validation_set_df.groupby('user_id')['item_id'].apply(set).to_dict()

            # TWORZENIE RZADKIEJ MACIERZY używając coo (bo większość komórek jest pusta) macierz kontrolna do walidacji + konwerscja na csr zoptymalizowany pod trening modelu ocen
            user_item_matrix_for_validation = coo_matrix(
                (my_train_set_df['rating'], (my_train_set_df['user_idx'], my_train_set_df['item_idx'])),
                shape=(num_users, num_items) ).tocsr()

            # Uruchamianie walidacji
            map_score = train_and_validate_als(
                params=best_params, val_users=users_to_evaluate_raw_ids, ground_truth=ground_truth_val,
                user_item_matrix=user_item_matrix_for_validation, user_to_idx=user_to_idx, idx_to_item=idx_to_item )

            print(f"\nWynik MAP@10 dla pojedynczego modelu: {map_score:.5f}")

    else:
        # Proces szkolenia modelu na więcej niż jednym zbiorze parametrów

        print("\n[ETAP 2/3] Podział danych do walidacji")

        # Znowu chronologicznie ogarniamy dane
        train_df_sorted = train_df.sort_values(by='timestamp').reset_index(drop=True)

        split_point = int(len(train_df_sorted) * VALIDATION_SPLIT_RATIO)

        # Tworzenie zbioru do trenowania modelu podczas walidacji
        my_train_set_df = train_df_sorted.iloc[:split_point]

        # Tworzenie zbioru do oceny modelu na podstawie HISTORYCZNYCH danych
        my_validation_set_df = train_df_sorted.iloc[split_point:]

        del train_df_sorted

        gc.collect()

        # Jest to po to aby rozwiązać problem z brakującymi użytkownikami którzy byli w walidacji ale w treningu i testach już nie

        # Tworzenie zbiorów aby sprawdzić rekomendacje użytkowników z produktami
        users_in_validation_raw_ids = my_validation_set_df['user_id'].unique()

        # Historyczni użytkownicy dla walidacji
        users_in_train_for_validation_raw_ids = my_train_set_df['user_id'].unique()

        # Użytkownicy zarówno z dwóch zbiorów
        users_to_evaluate_raw_ids_full = np.intersect1d(users_in_validation_raw_ids, users_in_train_for_validation_raw_ids)

        np.random.seed(ALS_RANDOM_STATE)

        # Wczytanie ilości użytkowników do walidacji
        num_users_to_sample = int(len(users_to_evaluate_raw_ids_full) * (VALIDATION_SAMPLE_PERCENT / 100.0))

        # Losowe wybranie użytkowników ze zbioru aby nie wybrać użytkownika jednego dwa razy
        users_to_evaluate_raw_ids = np.random.choice(users_to_evaluate_raw_ids_full, size=num_users_to_sample, replace=False)

        print(f"Zastosowano walidacji: {VALIDATION_SAMPLE_PERCENT}% użytkowników ({len(users_to_evaluate_raw_ids)} z {len(users_to_evaluate_raw_ids_full)})")

        # Kolejna jasna prawda dla walidacji
        ground_truth_val = my_validation_set_df.groupby('user_id')['item_id'].apply(set).to_dict()

        # Macierz 0 i 1 dla interakcji albo jej braku
        user_item_matrix_for_validation = coo_matrix( (my_train_set_df['rating'],
        (my_train_set_df['user_idx'], my_train_set_df['item_idx'])), shape=(num_users, num_items) ).tocsr()

        print("\n[ETAP 3/3] Uruchamianie pętli walidacyjnej")

        # Zapisy wyników prób
        results = []

        # Śledzimy najlepsze wyniki MAP@10
        best_map_score = -1.0

        # Walidacja na podstawie parametrów z configu
        for params in ALS_CONFIG:
            map_score = train_and_validate_als( params=params, val_users=users_to_evaluate_raw_ids, ground_truth=ground_truth_val,
                user_item_matrix=user_item_matrix_for_validation, user_to_idx=user_to_idx, idx_to_item=idx_to_item )

            # Zapamiętanie jakie parametry były użyte
            results.append({'params': params, 'map_score': map_score})

            # Sprawdzanie czy zwalidowany model nie jest najlepszy według MAP@10
            if map_score > best_map_score:
                best_map_score = map_score
                best_params = params
                print(f" -> Nowy najlepszy wynik MAP@10: {best_map_score:.5f} dla {params['name']} <-")

        print("Podsumowanie wszystkich prób:")

        # Wyświetlanie wszystkich wyników MAP@10 dla przejżystości
        for res in results: print(f" - {res['params']['name']}: MAP@10 = {res['map_score']:.5f}")

        # Wypisanie wyników
        print(f"\nNajlepszy wynik: {best_map_score:.5f}")
        print(f"Najlepsze parametry: {best_params}")

    # Patrzymy na wybrane najlepsze parametry i wtedy trenujemy najlepszy model końcowo
    if best_params:
        train_final_model_and_predict( best_params=best_params, full_train_df=train_df, test_df=test_df, user_to_idx=user_to_idx,
            idx_to_item=idx_to_item, num_users=num_users, num_items=num_items )

    else: print("Błąd z parametrami")

if __name__ == '__main__':
    main()

[ETAP 1/3] Wczytywanie i przetwarzanie danych
Całkowita liczba unikalnych użytkowników (zmapowanych): 868218
Całkowita liczba unikalnych przedmiotów (zmapowanych): 76747


C:\Users\bart_\AppData\Local\Programs\Python\Python312\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 20 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()



[ETAP 3/3] Trenujemy tylko jeden model z jednym zbiorem parametrów
[ETAP 2/3] Wyłączona walidacja danych

[ETAP 3/3 + Ostateczny trening] Trenowanie finalnego modelu i dawanie wyniku csv
Używam parametrów: factors=15, regularization=0.001, iterations=40


Generowanie csv: 100%|██████████| 412461/412461 [02:38<00:00, 2609.88it/s]



Plik submission zapisany do Dane/submission.csv
